In [ ]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# view = "tab_adult"
view = "texas_charges"

# LIM = 100000
enc = catalog.load(f'{view}.wrk.idx_table').sample()#[:LIM]
trn = catalog.load(f'{view}.trn.table')
attrs = trn['idx'].get_attributes()
len(enc)

8867653


In [ ]:
tst = catalog.load(f'{view}.wrk.table').sample()

In [ ]:
tst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9846386 entries, 0 to 12311820
Data columns (total 13 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   record_id              int64   
 1   revenue_code           category
 2   hcpcs_qualifier        category
 3   hcpcs_procedure_code   category
 4   modifier_1             category
 5   modifier_2             category
 6   modifier_3             category
 7   modifier_4             category
 8   unit_measurement_code  category
 9   units_of_service       Int32   
 10  unit_rate              float32 
 11  chrgs_line_item        float32 
 12  chrgs_non_cov          float32 
dtypes: Int32(1), category(8), float32(3), int64(1)
memory usage: 403.9 MB


In [ ]:
tst.columns


Index(['record_id', 'revenue_code', 'hcpcs_qualifier', 'hcpcs_procedure_code',
       'modifier_1', 'modifier_2', 'modifier_3', 'modifier_4',
       'unit_measurement_code', 'units_of_service', 'unit_rate',
       'chrgs_line_item', 'chrgs_non_cov'],
      dtype='object')


In [ ]:
import pandas as pd

tst.astype(
    {'record_id': 'int64', 'revenue_code': 'object', 'hcpcs_qualifier': 'object', 'hcpcs_procedure_code': 'object',
       'modifier_1': 'object', 'modifier_2': 'object', 'modifier_3': 'object', 'modifier_4': 'object',
       'unit_measurement_code': 'object', 'units_of_service': 'float64', 'unit_rate': 'float64',
       'chrgs_line_item': 'float64', 'chrgs_non_cov': 'float64'}
).info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9846386 entries, 0 to 12311820
Data columns (total 13 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   record_id              int64  
 1   revenue_code           object 
 2   hcpcs_qualifier        object 
 3   hcpcs_procedure_code   object 
 4   modifier_1             object 
 5   modifier_2             object 
 6   modifier_3             object 
 7   modifier_4             object 
 8   unit_measurement_code  object 
 9   units_of_service       float64
 10  unit_rate              float64
 11  chrgs_line_item        float64
 12  chrgs_non_cov          float64
dtypes: float64(4), int64(1), object(8)
memory usage: 3.4 GB


In [ ]:
from pasteur.marginal import expand_table, calc_marginal, AttrSelector, AttrSelectors

cols, cols_noncommon, domains = expand_table(attrs, enc)
for name in domains:
    print(f"{name:>25s}: {domains[name][0]:5d}")

             revenue_code:   391
          hcpcs_qualifier:     5
     hcpcs_procedure_code:  3383
               modifier_1:    47
               modifier_2:    12
               modifier_3:     3
               modifier_4:     3
    unit_measurement_code:     6
         units_of_service:    21
                unit_rate:    21
          chrgs_line_item:    20
            chrgs_non_cov:    20


In [ ]:
enc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8867653 entries, 0 to 11087787
Data columns (total 12 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   revenue_code           uint16
 1   hcpcs_qualifier        uint8 
 2   hcpcs_procedure_code   uint16
 3   modifier_1             uint8 
 4   modifier_2             uint8 
 5   modifier_3             uint8 
 6   modifier_4             uint8 
 7   unit_measurement_code  uint8 
 8   units_of_service       uint8 
 9   unit_rate              uint8 
 10  chrgs_line_item        uint8 
 11  chrgs_non_cov          uint8 
dtypes: uint16(2), uint8(10)
memory usage: 186.1 MB


In [ ]:
# tst = ['age', 'workclass', 'education', 'education-num']

tst = ['unit_measurement_code', 'chrgs_non_cov', "modifier_2", 'unit_rate']
# tst = ['unit_measurement_code', 'unit_measurement_code', 'unit_measurement_code']

dom = 1
for n in tst:
    dom *= domains[n][0]

print(dom)

30240


In [ ]:
domains


{
    'revenue_code': [391, 3],
    'hcpcs_qualifier': [5, 3],
    'hcpcs_procedure_code': [3383, 3],
    'modifier_1': [47, 3],
    'modifier_2': [12, 3],
    'modifier_3': [3, 3],
    'modifier_4': [3, 3],
    'unit_measurement_code': [6, 3],
    'units_of_service': [21, 2],
    'unit_rate': [21, 2],
    'chrgs_line_item': [20],
    'chrgs_non_cov': [20]
}


In [ ]:
import numpy as np

np.ravel_multi_index([enc[n] for n in tst], dims=[domains[n][0] for n in tst])

array([16633, 16633, 16633, ..., 26713, 26713, 26712])


In [ ]:
def ravel_manual(cols, dims):
    mul = dims[-1]
    idx = cols[-1]
    for col, dim in zip(cols[:-1:-1], dims[:-1:-1]):
        idx += mul*col
        mul *= dim
    return idx

ravel_manual([enc[n] for n in tst], dims=[domains[n][0] for n in tst])


charge_id
0           31304
1           31304
2           31304
4           31304
6           31304
            ...  
11087783     1105
11087784    31346
11087785    31346
11087786    31346
11087787     1105
Name: unit_rate, Length: 8867653, dtype: uint16


In [ ]:
x = AttrSelector(tst[0], 0, {tst[0]: 0})
p = {str(i): AttrSelector(col, 0, {col: 0}) for i, col in enumerate(tst[1:])}

%timeit calc_marginal(cols, cols_noncommon, domains, x, p)

80.2 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
import numpy as np

%timeit np.histogramdd(enc[tst].to_numpy(), [domains[col][0] for col in tst])

548 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%load_ext line_profiler

In [ ]:
Op = tuple[int, np.ndarray]

from pasteur.marginal import calc_marginal, calc_marginal_1way

%timeit calc_marginal(cols, cols_noncommon, domains, x, p)
# %lprun -f calc_marginal2 calc_marginal2(cols, cols_noncommon, domains, x, p)
b = calc_marginal(cols, cols_noncommon, domains, x, p)


22.5 ms ± 9.37 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
data = catalog.load(f'{view}.wrk.idx_table')

from pasteur.utils.progress import init_pool
from pasteur.marginal import MarginalOracle, MarginalRequest

reqs = [MarginalRequest(x, p, False) for _ in range(10000)]

In [ ]:
with init_pool():
    o = MarginalOracle(attrs, data)
    out = o.process(reqs)
    

           >>>>>>>  Processing partition:  18%|############3                                                       | 8/44 [00:00<00:00,  1.00it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_631413/1442494134.py:3 in <cell line: 1>                                          │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_631413/1442494134.py'                       │
│                                                                                                  │
│ /srv/data/antheas/projects/pasteur/src/pasteur/marginal/__init__.py:130 in process               │
│                                                                                                  │
│   127 │   │   for batch in piter(                                                                │
│   128 │   │   │   self.data.values(), total=len(self.data), desc="Processing partition"          │
│   129 │   │   ):                                                                                 │
│ ❱ 130 │   │   │   old = self.process_batch(batch, requests, old)                                 │
│   131 │   │   return old                                                                         │
│   132 │                                                                                          │
│   133 │   def close(self):                                                                       │
│                                                                                                  │
│ /srv/data/antheas/projects/pasteur/src/pasteur/marginal/__init__.py:109 in process_batch         │
│                                                                                                  │
│   106 │   │   else:                                                                              │
│   107 │   │   │   per_call_args = [{"req": req} for req in requests]                             │
│   108 │   │                                                                                      │
│ ❱ 109 │   │   res = process_in_parallel(                                                         │
│   110 │   │   │   _marginal_worker, per_call_args, base_args, 100, "Calculating marginals"       │
│   111 │   │   )                                                                                  │
│   112                                                                                            │
│                                                                                                  │
│ /srv/data/antheas/projects/pasteur/src/pasteur/utils/progress.py:355 in process_in_parallel      │
│                                                                                                  │
│   352 │   pbar = piter(desc=desc, leave=False, total=n_tasks)                                    │
│   353 │   n = 0                                                                                  │
│   354 │   while not res.ready():                                                                 │
│ ❱ 355 │   │   u = progress_queue.get()                                                           │
│   356 │   │   n += u                                                                             │
│   357 │   │   pbar.update(u)                                                                     │
│   358 │   │   if n == n_tasks:                                                                   │
│ <string>:2 in get                                                                                │
│                                                                                                  │
│ /usr/lib/python3.10/multiprocessing/managers.py:818 in _callmethod                               │
│                                                                                                  │
│    815 │   │   │   conn = self._tls.connection                                                   │
│    816 │   │                                               

In [ ]:
del out